In [1]:
using CSV
using DataFrames
using StatsBase  # For standardization

In [2]:
# Load the dataset
file_path = "bank-additional-full.csv"
df = CSV.read(file_path)

,age,job,marital,education,default,housing,loan
,Int64,String,String,String,String,String,String
1,56,housemaid,married,basic.4y,no,no,no
2,57,services,married,high.school,unknown,no,no
3,37,services,married,high.school,no,yes,no
4,40,admin.,married,basic.6y,no,no,no
5,56,services,married,high.school,no,no,yes
6,45,services,married,basic.9y,unknown,no,no
7,59,admin.,married,professional.course,no,no,no
8,41,blue-collar,married,unknown,unknown,no,no
9,24,technician,single,professional.course,no,yes,no


In [3]:
# Convert dataframe to matrix
m = convert(Matrix, df)
# Categorical feature vectors which need mapping to numbers
marital_vector = Array{String,1}(undef, size(m)[1])
default_vector = Array{String,1}(undef, size(m)[1])
loan_vector = Array{String,1}(undef, size(m)[1])
month_vector = Array{String,1}(undef, size(m)[1])
duration_vector = Array{Int64,1}(undef, size(m)[1])
poutcome_vector = Array{String,1}(undef, size(m)[1])
job_vector = Array{String,1}(undef, size(m)[1])
education_vector = Array{String,1}(undef, size(m)[1])
housing_vector = Array{String,1}(undef, size(m)[1])
contact_vector = Array{String,1}(undef, size(m)[1])
dow_vector = Array{String,1}(undef, size(m)[1])
y_vector = Array{String,1}(undef, size(m)[1])

41188-element Array{String,1}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
   ⋮   
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [4]:
# Map categorical features to numbers
for i in 1:size(m)[1]
    for j in 1:size(m)[2]
        # Check if columns match
        if j == 3
            marital_vector[i] = m[i,j]
        elseif j == 5
            default_vector[i] = m[i,j]
        elseif j == 7
            loan_vector[i] = m[i,j]
        elseif j == 9
            month_vector[i] = m[i,j]
        elseif j == 11
            duration_vector[i] = m[i,j]
        elseif j == 15
            poutcome_vector[i] = m[i,j]
        elseif j == 2
            job_vector[i] = m[i,j]
        elseif j == 4
            education_vector[i] = m[i,j]
        elseif j == 6
            housing_vector[i] = m[i,j]
        elseif j == 8
            contact_vector[i] = m[i,j]
        elseif j == 10
            dow_vector[i] = m[i,j]
        elseif j == 21
            y_vector[i] = m[i,j]
        end
    end
end

In [5]:
function map_to_int(arr, T, initial_count=nothing)
    # Dynamically map the strings to numbers
    unique_list = T[]
    for v in arr
        if v in unique_list
            continue
        else
            push!(unique_list, v)
        end
    end
    cleaned_list = Array{Int64, 1}(undef, length(arr))
    if initial_count == nothing
        count = 1
    else
        # Works since dataset has "no" as first and not "yes",
        # not the best way but is suitable for now
        count = initial_count
    end
    for i in unique_list
        for j in 1:length(arr)
            if arr[j] == i
                cleaned_list[j] = count
            end
        end
        count += 1
    end
    return cleaned_list

end

map_to_int (generic function with 2 methods)

In [6]:
cleaned_marital = map_to_int(marital_vector, String)
cleaned_default = map_to_int(default_vector, String)
cleaned_loan = map_to_int(loan_vector, String)
cleaned_month = map_to_int(month_vector, String)

cleaned_duration = map_to_int(duration_vector, Int64)
cleaned_poutcome = map_to_int(poutcome_vector, String)
cleaned_job = map_to_int(job_vector, String)
cleaned_education = map_to_int(education_vector, String)

cleaned_housing = map_to_int(housing_vector, String)
cleaned_contact = map_to_int(contact_vector, String)
cleaned_dow = map_to_int(dow_vector, String)
# Special case where we want only 1 and 0 
y = map_to_int(y_vector, String, 0)

41188-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 1
 0

In [7]:
# Update the matrix with the numerical mapped equivalents
for i in 1:size(m)[1]
    for j in 1:size(m)[2]
        if j == 3
            m[i,j] = cleaned_marital[i]
        elseif j == 5
            m[i,j] = cleaned_default[i]
        elseif j == 7
            m[i,j] = cleaned_loan[i]
        elseif j == 9
            m[i,j] = cleaned_month[i]
        elseif j == 11
            m[i,j] = cleaned_duration[i]
        elseif j == 15
            m[i,j] = cleaned_poutcome[i]
        elseif j == 2
            m[i,j] = cleaned_job[i]
        elseif j == 4
            m[i,j] = cleaned_education[i]
        elseif j == 6
            m[i,j] = cleaned_housing[i]
        elseif j == 8
            m[i,j] = cleaned_contact[i]
        elseif j == 10
            m[i,j] = cleaned_dow[i]
        end
    end
end

In [8]:
# Remove y from feature matrix
x = Matrix{Float64}(undef, size(m)[1], size(m)[2]-1)

for i in 1:size(m)[1]
    for j in 1:size(m)[2]-1
        # Merge the outcome value as the last index
        if j == size(m)[2]-1
            x[i,j] = y[i]
            continue
        end
        x[i,j] = m[i,j]
    end
end

In [9]:
# Concatenate a vector of 1 to feature matrix to represent bias
#x = hcat(x, ones(Int64, size(x)[1]))

In [10]:
# Standardize x
m_fit = fit(ZScoreTransform, x, dims=2)
x = StatsBase.transform(m_fit, x)

41188×20 Array{Float64,2}:
 -0.00236248   -0.248664  -0.248664  …  -0.416149  -0.231391  -0.253142
  0.000996409  -0.245362  -0.249842     -0.417366  -0.232565  -0.254321
 -0.0843215    -0.241073  -0.245552     -0.413053  -0.228278  -0.250031
 -0.0720173    -0.237754  -0.246713     -0.414241  -0.229436  -0.251192
 -0.00393143   -0.245852  -0.250332     -0.417884  -0.233052  -0.254812
 -0.0514244    -0.244083  -0.248563  …  -0.416132  -0.231282  -0.253044
  0.00771871   -0.243231  -0.252193     -0.419792  -0.234909  -0.256674
 -0.069813     -0.235634  -0.249079     -0.416692  -0.231793  -0.25356 
 -0.142595     -0.227719  -0.241159     -0.4132    -0.228359  -0.25012 
 -0.137184     -0.240204  -0.240204     -0.412201  -0.227407  -0.249162
 -0.0704958    -0.236341  -0.249788  …  -0.417427  -0.2325    -0.254271
 -0.137645     -0.240674  -0.240674     -0.412688  -0.227876  -0.249633
 -0.121329     -0.233348  -0.24231      -0.414372  -0.229508  -0.251271
  ⋮                                  

In [11]:
# Split into testing and training
TRAIN_PERCENT = 0.8
# Calculate number of rows to use as train sample
training_row_length = trunc(Int64, size(x)[1] * TRAIN_PERCENT)
training_x = x[1:training_row_length-1, :]
testing_x = x[training_row_length: size(x)[1], :]

training_y = y[1:training_row_length-1, :]
testing_y = y[training_row_length: size(y)[1], :]

8239×1 Array{Int64,2}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 1
 0

In [12]:
function hypothesis(v_theta, x)
    z = transpose(v_theta) * x
    # Sigmoid
    return 1/(1+exp(-z))
end

hypothesis (generic function with 1 method)

In [13]:
function sum_square_theta(theta)
    res = 0
    for i in 1:size(theta)[1]
        res += theta[i]^2
    end
    return res
end

sum_square_theta (generic function with 1 method)

In [14]:
function cost_function(X, Y, theta, lambda)
    m = size(X)[1]
    cross_result = 0 # result of cross-entropy function
    for i in 1:m
        y = Y[i]
        x = X[i, :]
        cross_result += (1-y)*log(hypothesis(theta, x))
#         println((1-y)*log(hypothesis(theta, x)))
    end
    # regularize
#     println(cross_result)
    cross_result += (lambda/2*m)*sum_square_theta(theta)
#     println(cross_result)
    return -(1/m)*cross_result
end
cost_function(training_x, training_y, zeros(size(training_x)[2]), 0.2)

0.6489695399886286

In [21]:
function update_theta(X, Y, theta, lr, lambda)
    m = size(X)[1]
    for j in 1:size(theta)[1]
        if j == 1
            temp1 = 0 # summation result when theta index 0
            for i in 1:m
                x = X[i, :]
                y = Y[i:i, :][1]
                temp1 += hypothesis(theta, x) - y
                temp1 *= x[1]
            end
            theta[1] = theta[1] - lr*(1/m)*temp1
        else
            temp2 = 0 # summation result when theta index not 0
            for i in 1:m
                x = X[i, :]
                y = Y[i:i, :][1]
                temp2 += hypothesis(theta, x) - y
                temp2 *= x[j]
                temp2 += (lambda/m) * theta[j]
            end
            theta[j] = theta[j] - lr*(1/m)*temp2
        end
    end
    return theta
end
# update_theta(training_x, training_y,zeros(size(training_x)[2]), 0.01, 0.02)

20-element Array{Float64,1}:
   2.448537299723549e-8 
   3.654113917772598e-8 
   3.766700222593438e-8 
   3.798581416024118e-8 
   3.805329940403197e-8 
   3.803457958852789e-8 
   3.822804761800197e-8 
   3.7678226864746915e-8
   3.80576386024865e-8  
   3.8057638361898347e-8
  -1.1771485628040417e-6
   3.747099413860814e-8 
 Inf                    
   7.686867266219111e-8 
   7.6115297313212e-8   
   7.821112372911533e-8 
  -2.4026835438140403e-8
   1.0639706279706901e-7
   7.588896550763635e-8 
   7.686865377610943e-8 